In [37]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline  
pd.set_option('display.max_columns', None)
from sklearn.linear_model import SGDClassifier
import sklearn.cross_validation
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics 

In [40]:
TITLE = ['Position', 'HistCTR', 'IsClick', 'IsUserLoggedOn', 'Price']
SOURCE = "../data/train_sampled.csv"

In [2]:
df = pd.read_csv(SOURCE , sep=',' , usecols=TITLE) 
label_frame = df['IsClick']
df = df.drop(['IsClick'],axis=1)

In [16]:
df.describe()

,Position,HistCTR,IsUserLoggedOn,Price
count,3438867.000000,3438867.000000,3438867.000000,3.437818e+06
mean,3.385109,0.012274,0.307141,1.889284e+04
std,2.936309,0.016786,0.461308,1.484530e+05
min,1.000000,0.000010,0.000000,0.000000e+00
25%,1.000000,0.002967,0.000000,1.390000e+03
50%,1.000000,0.006735,0.000000,4.429000e+03
75%,7.000000,0.014599,1.000000,1.300000e+04
max,7.000000,1.000000,1.000000,1.300000e+08


In [21]:
df[df.Price.isnull()]
prices_median = df.Price.median()

In [22]:
df.Price.fillna(prices_median, inplace=True)

In [27]:
df.Price.head()

0     1000
1     3000
2     5470
3    10419
4      878
Name: Price, dtype: float64

In [61]:
scaler = MinMaxScaler()
df.Price = scaler.fit_transform(df.Price.reshape(-1, 1))

In [62]:
df.Position = scaler.fit_transform(df.Position.reshape(-1, 1))

In [31]:
clf = SGDClassifier(loss="log")
scores = cross_val_score(clf, df, label_frame, cv=10, scoring='log_loss')
scores.mean()

-0.62318730597876859

In [34]:
clf.fit(df, label_frame)
clf.predict_proba(df)
clf.score(df, label_frame )

0.66758295682851354

In [39]:
sklearn.metrics.log_loss(label_frame, clf.predict_proba(df))

0.62316994215069021

In [ ]:
inc_clf = SGDClassifier(loss="log")
scaler = MinMaxScaler()
for chunk in pd.read_csv(SOURCE, sep=',', usecols=TITLE , chunksize=10000):
    Y = chunk['IsClick']
    X = chunk.drop(['IsClick'],axis=1)
    X.Price.fillna(X.Price.median(), inplace=True)
    X.Price = scaler.fit_transform(X.Price.reshape(-1, 1))
    #X.Position = scaler.fit_transform(X.Position.reshape(-1, 1))
    inc_clf.partial_fit(X, Y ,  classes=[0,1])
    

In [64]:
sklearn.metrics.log_loss(label_frame, inc_clf.predict_proba(df))

0.73899191921588725